In [1]:
import numpy as np

In [2]:
def cholesky_decomposition(A):
    L = np.zeros_like(A)
    n = A.shape[0]
    
    for j in range(n):
        L[j, j] = np.sqrt(A[j, j] - np.sum(L[j, 0:j] ** 2))
        for i in range(j + 1, n):
            L[i, j] = (A[i, j] - np.sum(L[j, 0:j] * L[i, 0:j])) / L[j, j]

    return L

In [3]:
def solve_upper_triangle(A, b):
    x = np.zeros((A.shape[0]))
    for i in range(A.shape[0] - 1, -1, -1):
        x[i] = (b[i] - A[i, i + 1:].dot(x[i + 1:])) / A[i, i]
    return x


def solve_lower_triangle(A, b):
    x = np.zeros((A.shape[0]))
    for i in range(0, A.shape[0]):
        x[i] = (b[i] - A[i, :i].dot(x[:i])) / A[i, i]
    return x

In [4]:
def cholesky_solve(A, b):
    L = cholesky_decomposition(A)
    y = solve_lower_triangle(L, b)
    return solve_upper_triangle(L.T, y)

In [5]:
def cholesky_lstsq(A, b):
    # Build symmetric, positive-definite matrix
    C = A.T.dot(A)
    L = cholesky_decomposition(C)
    y = solve_lower_triangle(L, A.T.dot(b))
    return solve_upper_triangle(L.T, y)

In [12]:
def test_solve(sizes):
#     size = 20
    
    for size in sizes:
        # Generate random symmetric, positive-definite matrix
        A = np.random.uniform(0, 100, (size, size))
        A = A.T.dot(A)
        b = np.random.uniform(0, 100, size)

        # Solve using cholesky decomposition
        x = cholesky_solve(A, b)

        # Solve using numpy
        x_true = np.linalg.solve(A, b)

        # Compare results
        print('\nSize: ', size, 'Error: ', np.linalg.norm(x - x_true))


def test_lstsq(sizes):
#     size = (17, 10)
    
    for size in sizes:
        # Generate random matrix
        A = np.random.uniform(0, 100, size)
        b = np.random.uniform(0, 100, size[0])

        # Solve using cholesky decomposition
        x = cholesky_lstsq(A, b)

        # Solve using numpy
        x_true = np.linalg.lstsq(A, b, rcond=None)[0]

        # Compare results
        print('\nSize: ', size, 'Error: ', np.linalg.norm(x - x_true))

In [13]:
print('Solve linear system:')
test_solve([3, 5, 10, 20, 50, 100])

print('\nSolve least squares problem:')
test_lstsq([(5, 3), (20, 17)])

Solve linear system:

Size:  3 Error:  1.0103182026100663e-16

Size:  5 Error:  2.739844181681855e-13

Size:  10 Error:  1.57377388878087e-10

Size:  20 Error:  1.212858309640873e-13

Size:  50 Error:  5.143516853461974e-08

Size:  100 Error:  7.209777634860486e-11

Solve least squares problem:

Size:  (5, 3) Error:  6.704249325346046e-16

Size:  (20, 17) Error:  4.956489995302799e-14
